In [1]:
DATA_DIR = '/kaggle/input/learn-together'
DATA_DIR = 'data'

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, KBinsDiscretizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [17]:
train_df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
test_df = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

train_df = train_df.set_index('Id')
X_train = train_df.drop(['Cover_Type'], axis=1)
y_train = train_df[['Cover_Type']]

X_test = test_df.set_index('Id')

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [19]:
mm = MinMaxScaler((0,1))
kb = KBinsDiscretizer(encode='ordinal', n_bins=20, strategy='quantile')
pca = PCA(random_state=60, n_components=47)
knn = KNeighborsClassifier(algorithm='ball_tree', n_jobs=-1, n_neighbors=1)

pipe = Pipeline(steps=[
        ('scale', mm),
        ('pca', pca),
        ('disc', kb),
        ('pred', knn)
])

pipe.fit(X_train, y_train)

/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/sklearn/pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


Pipeline(memory=None,
         steps=[('scale', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('pca',
                 PCA(copy=True, iterated_power='auto', n_components=47,
                     random_state=60, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('disc',
                 KBinsDiscretizer(encode='ordinal', n_bins=20,
                                  strategy='quantile')),
                ('pred',
                 KNeighborsClassifier(algorithm='ball_tree', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=-1, n_neighbors=1, p=2,
                                      weights='uniform'))],
         verbose=False)

In [28]:
print(pipe.score(X_train, y_train))
y_pred1 = pipe.predict(X_val)
accuracy_score(y_val, y_pred1)


1.0


0.8412698412698413

In [30]:
rf = RandomForestClassifier(
    **{'n_estimators': 200, 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1}
)
rf.fit(X_train, y_train)
print(rf.score(X_train, y_train))
y_pred2 = rf.predict(X_val)
accuracy_score(y_val, y_pred2)


/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


1.0


0.857473544973545

In [85]:
gbc = GradientBoostingClassifier()

gbc.fit(X_train, y_train)

/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [86]:
print(gbc.score(X_train, y_train))
y_pred3 = gbc.predict(X_val)
accuracy_score(y_val, y_pred3)


0.8416005291005291


0.7979497354497355

In [72]:
def report(y_true, y_pred):
    print('Accuracy: %s' % accuracy_score(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

In [73]:
report(y_val, y_pred1)

Accuracy: 0.8412698412698413
              precision    recall  f1-score   support

           1       0.77      0.70      0.73       423
           2       0.75      0.70      0.72       447
           3       0.86      0.80      0.83       438
           4       0.92      0.96      0.94       427
           5       0.85      0.93      0.89       420
           6       0.81      0.85      0.83       434
           7       0.92      0.95      0.93       435

    accuracy                           0.84      3024
   macro avg       0.84      0.84      0.84      3024
weighted avg       0.84      0.84      0.84      3024

[[294  76   0   0  19   3  31]
 [ 70 312   7   0  40  12   6]
 [  0   8 352  20   4  54   0]
 [  0   0   8 412   0   7   0]
 [  2  13   5   0 392   8   0]
 [  0   4  39  17   6 368   0]
 [ 17   4   0   0   0   0 414]]


In [74]:
report(y_val, y_pred2)

Accuracy: 0.857473544973545
              precision    recall  f1-score   support

           1       0.76      0.74      0.75       423
           2       0.79      0.70      0.74       447
           3       0.85      0.82      0.84       438
           4       0.93      0.98      0.95       427
           5       0.89      0.94      0.91       420
           6       0.83      0.86      0.85       434
           7       0.93      0.96      0.94       435

    accuracy                           0.86      3024
   macro avg       0.86      0.86      0.86      3024
weighted avg       0.85      0.86      0.86      3024

[[315  64   1   0  15   2  26]
 [ 81 311  10   0  32   8   5]
 [  0   1 361  20   3  53   0]
 [  0   0   3 419   0   5   0]
 [  0  11   6   0 396   7   0]
 [  0   5  42  12   1 374   0]
 [ 18   0   0   0   0   0 417]]


In [87]:
report(y_val, y_pred3)

Accuracy: 0.7979497354497355
              precision    recall  f1-score   support

           1       0.71      0.68      0.69       423
           2       0.76      0.60      0.67       447
           3       0.75      0.75      0.75       438
           4       0.91      0.97      0.94       427
           5       0.80      0.91      0.85       420
           6       0.74      0.74      0.74       434
           7       0.90      0.94      0.92       435

    accuracy                           0.80      3024
   macro avg       0.79      0.80      0.79      3024
weighted avg       0.79      0.80      0.79      3024

[[289  71   1   0  16   5  41]
 [ 96 266   9   0  59  13   4]
 [  0   1 330  23   6  78   0]
 [  0   0   7 416   0   4   0]
 [  1  13  12   0 381  13   0]
 [  0   1  80  18  14 321   0]
 [ 23   0   0   0   2   0 410]]
